In [3]:
import numpy as np
import networkx as nx
from karateclub import BoostNE
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [4]:
import os
files = os.listdir('../graph-data/seattle-graphs/original/')

In [12]:
embeddings = []
for file in tqdm(files):
    G = nx.read_gpickle('../graph-data/seattle-graphs/original/'+file)
    model = BoostNE(dimensions=16, iterations=15)
    model.fit(G)
    embeddings.append(model.get_embedding())
    break

  0%|                                                  | 0/8623 [00:00<?, ?it/s]


In [21]:
import torch
import torch.nn as nn
import torch.functional as f

In [33]:
lr1 = nn.Linear(256,128)
lr2 = nn.Linear(128,64)
relu = nn.ReLU()
sig = nn.Sigmoid()
inputs = torch.from_numpy(embeddings[0]).float()
output = lr2(relu(lr1(inputs)))

In [29]:
inputs.shape, output.shape

(torch.Size([97, 256]), torch.Size([97, 64]))

In [52]:
A_pred = torch.matmul(output, output.T)
A_pred.shape

torch.Size([97, 97])

In [100]:
A_pred

tensor([[0.1951, 0.1952, 0.1953,  ..., 0.1969, 0.1961, 0.1919],
        [0.1952, 0.1954, 0.1954,  ..., 0.1968, 0.1959, 0.1917],
        [0.1953, 0.1954, 0.1957,  ..., 0.1973, 0.1964, 0.1923],
        ...,
        [0.1969, 0.1968, 0.1973,  ..., 0.2002, 0.1987, 0.1946],
        [0.1961, 0.1959, 0.1964,  ..., 0.1987, 0.1984, 0.1938],
        [0.1919, 0.1917, 0.1923,  ..., 0.1946, 0.1938, 0.1914]],
       grad_fn=<MmBackward>)

In [101]:
A_ori = torch.from_numpy(nx.to_numpy_array(G))
A_ori

tensor([[1., 1., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], dtype=torch.float64)

In [79]:
bcrp = nn.BCELoss()

In [188]:
from torch.utils.data import Dataset, DataLoader

In [203]:
class GraphDataset(Dataset):
    def __init__(self, graph_dir, graph_files):
        
        # directory 
        self.graph_dir = graph_dir
        
        # all graph files
        self.graph_files = graph_files        
    
    def __len__(self):
        return len(self.graph_files)

    def __getitem__(self, idx):
        
        # single graph
        file = self.graph_files[idx]
        
        # graph paths
        graph_path = os.path.join(self.graph_dir, 'original', file)
        graph_del_path = os.path.join(self.graph_dir, 'deletion', file)
        graph_ins_path = os.path.join(self.graph_dir, 'insertion', file)
        
        # load graph
        graph = torch.from_numpy(nx.to_numpy_array(nx.read_gpickle(graph_path))).float().unsqueeze_(0)
        graph_del = torch.from_numpy(nx.to_numpy_array(nx.read_gpickle(graph_del_path))).float().unsqueeze_(0)
        graph_ins = torch.from_numpy(nx.to_numpy_array(nx.read_gpickle(graph_ins_path))).float().unsqueeze_(0)
        
        return graph, graph_del, graph_ins

In [ ]:
def collate_fn(data):
    """
       data: is a list of tuples with (example, label, length)
             where 'example' is a tensor of arbitrary shape
             and label/length are scalars
    """
    graph, graph_del, graph_ins = zip(*data)
    max_len = max(lengths)
    n_ftrs = data[0][0].size(1)
    features = torch.zeros((len(data), max_len, n_ftrs))
    labels = torch.tensor(labels)
    lengths = torch.tensor(lengths)

    for i in range(len(data)):
        j, k = data[i][0].size(0), data[i][0].size(1)
        features[i] = torch.cat([data[i][0], torch.zeros((max_len - j, k))])

    return features.float(), labels.long(), lengths.long()

In [205]:
a1,a2,a3 = graph.__getitem__(0)
b1,b2,b3 = graph.__getitem__(1)
c1,c2,c3 = graph.__getitem__(2)

In [206]:
torch.([a1,b1,c1])

RuntimeError: stack expects each tensor to be equal size, but got [1, 97, 97] at entry 0 and [1, 55, 55] at entry 1

In [204]:
graph= GraphDataset('../graph-data/seattle-graphs/', files)
train_dataloader = DataLoader(graph, batch_size=10, shuffle=True)